In [4]:
from ltc.trainer import FileBinaryClassifierTrainer

In [ ]:
pos_data = './data/pos'
neg_data = './data/neg'
trainer = FileBinaryClassifierTrainer(
    model_name='logistic_regression_example',
    max_length=4000,
    num_epochs=1000
)
model = trainer.train(pos_data, neg_data)
trainer.save(model)